# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import json

!pip install hvplot geoviews



# Import API key
from api_keys import geoapify_key

geoapify_key = "d5a811a0280148fbad47bda86e077f8e"

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jiuquan,39.7432,98.5174,66.83,23,26,6.22,CN,1686074414
1,1,edinburgh of the seven seas,-37.0676,-12.3116,54.50,66,100,16.69,SH,1686074415
2,2,adamstown,-25.0660,-130.1015,69.87,57,4,16.87,PN,1686074415
3,3,atafu village,-8.5421,-172.5159,83.21,74,10,10.33,TK,1686074415
4,4,papao,-17.7500,-149.5500,74.88,72,21,15.08,PF,1686074415


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(x="Lng", y="Lat", c="Humidity", geo=True, title="City Map" )

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values


# Drop any rows with null values
ideal_cities_df = city_data_df[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 10)].dropna()

# Display sample data
ideal_cities_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,ilha de mocambique,-15.0342,40.7358,78.69,74,0,8.81,MZ,1686074426
88,88,lincoln city,44.9582,-124.0179,77.43,55,0,7.00,US,1686074427
104,104,myrina,39.8797,25.0742,72.64,56,0,9.22,GR,1686074419
113,113,colonia,50.9333,6.9500,75.96,39,0,8.05,DE,1686074213
133,133,cidade velha,14.9167,-23.6167,79.27,69,0,5.75,CV,1686074274


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,ilha de mocambique,-15.0342,40.7358,78.69,74,0,8.81,MZ,1686074426,
88,88,lincoln city,44.9582,-124.0179,77.43,55,0,7.00,US,1686074427,
104,104,myrina,39.8797,25.0742,72.64,56,0,9.22,GR,1686074419,
113,113,colonia,50.9333,6.9500,75.96,39,0,8.05,DE,1686074213,
133,133,cidade velha,14.9167,-23.6167,79.27,69,0,5.75,CV,1686074274,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "type": "lodging",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter_param = {
        "distance": {
            "geometry": "point",
            "coordinates": [lon, lat],
            "lt": radius
        }
    }
    
    # Convert the filter object to a JSON string
    filter_param_str = json.dumps(filter_param)
    
    # Add the filter parameter to the params dictionary
    params["filter"] = filter_param_str
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    print(name_address)
    
 #     Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    

# Display sample data
hotel_df

Starting hotel search
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
ilha de mocambique - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
lincoln city - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
myrina - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
colonia - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
cidade velha - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
lemmon valley - nearest hotel: No hotel found
{'statusCode': 400, 'error': 'Bad Request', 'message

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,ilha de mocambique,-15.0342,40.7358,78.69,74,0,8.81,MZ,1686074426,No hotel found
88,88,lincoln city,44.9582,-124.0179,77.43,55,0,7.00,US,1686074427,No hotel found
104,104,myrina,39.8797,25.0742,72.64,56,0,9.22,GR,1686074419,No hotel found
113,113,colonia,50.9333,6.9500,75.96,39,0,8.05,DE,1686074213,No hotel found
133,133,cidade velha,14.9167,-23.6167,79.27,69,0,5.75,CV,1686074274,No hotel found
161,161,lemmon valley,39.6360,-119.8432,72.41,45,0,5.75,US,1686074439,No hotel found
196,196,nova londrina,-22.7658,-52.9850,76.06,52,0,8.81,BR,1686074266,No hotel found
294,294,lindi,-10.0000,39.7167,71.49,89,0,3.94,TZ,1686074456,No hotel found
327,327,archangelos,36.2149,28.1149,74.30,76,0,4.07,GR,1686074461,No hotel found
391,391,saint-jean-de-monts,46.7833,-2.0667,72.70,54,0,9.57,FR,1686074470,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:


# Configure the map plot
map_plot = hotel_df.hvplot.points(x="Lng", y="Lat", c="Humidity", hover_cols=["City", "Country", "Hotel Name"], size="Humidity", geo=True, title="City Map with Hotels")

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)